In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvtk

In [ ]:
# modelのインポート
df_model = pd.read_csv("model_data.csv")  # モデルデータ
df_nodes = pd.read_csv("node_data.csv")     # ノードデータ

node_num = max(df_model["tri_node_1"].max(), df_model["tri_node_2"].max(), df_model["tri_node_3"].max())
dof_node = 2
dof_total = node_num * dof_node
node_tria = 3
dof_tria3 = node_tria * dof_node
elemnt_num = max(df_model["element_no"])


print("=============解析条件=============")
print(f"接点数　  ： {node_num}")
print(f"要素数　  ： {elemnt_num}")
print(f"接点自由度： {dof_node}")
print(f"要素自由度： {dof_tria3}")
print(f"全自由度  ： {dof_total}")
print("==================================")

In [ ]:
# 物性値
NU = 0.3               # ポアソン比
YOUNG_MODULE = 210000  # ヤング率
t = 1                  # 板厚


# 平面歪み仮定のDマトリクス
d_coef = YOUNG_MODULE / ((1 - 2 * NU) * (1 + NU))

d_mat = d_coef * np.array([
    [1-NU, NU, 0],
    [NU, 1-NU, 0],
    [0, 0, (1-2*NU)/2]
])

In [ ]:
# Bマトリクス作成

b_mat = [None]
tri_arias = [None]

for element_no in df_model["element_no"].tolist():
    x_1 = float(df_model[df_model["element_no"] == element_no]["tri_node_1_x"])
    y_1 = float(df_model[df_model["element_no"] == element_no]["tri_node_1_y"])
    x_2 = float(df_model[df_model["element_no"] == element_no]["tri_node_2_x"])
    y_2 = float(df_model[df_model["element_no"] == element_no]["tri_node_2_y"])
    x_3 = float(df_model[df_model["element_no"] == element_no]["tri_node_3_x"])
    y_3 = float(df_model[df_model["element_no"] == element_no]["tri_node_3_y"])

    tri_aria = (x_1 * y_2 - x_1 * y_3 + x_2 * y_3 - x_2 * y_1 + x_3 * y_1 - x_3 * y_2) / 2
    coef = 1 / (2 * tri_aria)
    
    b_mat.append(
        np.array([
            [coef * (y_2 - y_3), 0, coef * (y_3 - y_1), 0, coef * (y_1 - y_2), 0],
            [0, coef * (x_3 - x_2), 0, coef * (x_1 - x_3), 0, coef * (x_2 - x_1)],
            [coef * (x_3 - x_2), coef * (y_2 - y_3), coef * (x_1 - x_3), coef * (y_3 - y_1), coef * (x_2 - x_1), coef * (y_1 - y_2)]
        ])
    )
    tri_arias.append(tri_aria)


In [ ]:
# 剛性マトリクスKeの作成(要素ごと)

ke_mat = [None]

for element_no in df_model["element_no"].tolist():

    ke_mat.append(
        t * tri_arias[element_no] * b_mat[element_no].T @ d_mat.T @ b_mat[element_no]
    )
    

In [ ]:
# 全体剛性マトリクスKの作成

k_mat = np.zeros((dof_total, dof_total))  # Kマトリクス, 0で初期化

for element_no in df_model["element_no"].tolist():
    
    for r in range(dof_tria3):
        # 要素内接点順(1, 2, 3)を求める
        er = math.ceil((r + 1) / dof_node)
        
        # 要素番号と要素内接点順からトータル接点番号を求める
        nr = 0
        if er == 1:
            nr = int(df_model[df_model["element_no"] == element_no]["tri_node_1"])
        elif er == 2:
            nr = int(df_model[df_model["element_no"] == element_no]["tri_node_2"])
        elif er == 3:
            nr = int(df_model[df_model["element_no"] == element_no]["tri_node_3"])

        # x成分かy成分か判断
        mr = (r + 1) % dof_node  # x(偶数)なら1, y(奇数)なら0
        
        # 全体合成マトリクスの番号を求める
        rt = nr * dof_node - mr - 1  # rt: Kマトリクスの行番号(np.array合わせで0から)
        
        for c in range(dof_tria3):
            # 要素内接点順(1, 2, 3)を求める
            ec = math.ceil((c + 1) / dof_node)
            
            # 要素番号と要素内接点順からトータル接点番号を求める
            nc = 0
            if ec == 1:
                nc = int(df_model[df_model["element_no"] == element_no]["tri_node_1"])
            elif ec == 2:
                nc = int(df_model[df_model["element_no"] == element_no]["tri_node_2"])
            elif ec == 3:
                nc = int(df_model[df_model["element_no"] == element_no]["tri_node_3"])

            # x成分かy成分か判断
            mc = (c + 1) % dof_node  # x(偶数)なら1, y(奇数)なら0
        
            # 全体合成マトリクスの番号を求める        
            ct = nc * dof_node - mc - 1  # ct: Kマトリクスの列番号(np.array合わせで0から)

            # KマトリクスにKeマトリクス足しこむ(加算する)
            k_mat[ct, rt] += ke_mat[element_no][c, r]


In [ ]:
# 境界条件(Boundary condition)の設定

f_vec = np.zeros(dof_total)  # 全体荷重ベクトル
u_vec = np.zeros(dof_total)  # 全体変位ベクトル
um_vec = []                  # 拘束目印ベクトル(拘束部分をTrueにする)

# ここで境界条件の設定を行う=======================
f_vec[5] = -100

for i in range(elemnt_num):
    
    if i == 0:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 3:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 25:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 26:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 27:
        um_vec.append(True)
        um_vec.append(True)
    else:
        um_vec.append(False)
        um_vec.append(False)


# =============================================


# Kマトリクスのコピーを作成
kc_mat = k_mat.copy()

for r in range(node_num * dof_node):  # 行方向に順に処理

    if um_vec[r]:  # 拘束条件が設定されている成分に対して処理
        print(f"{r}行ベクトル拘束条件設定")
        
        for rr in range(dof_total):  # 変位拘束成分以外の荷重ベクトルを修正
            if rr != r:
                f_vec[rr] = f_vec[rr] - kc_mat[r, rr] * u_vec[r]
    
        # 拘束条件の存在する行の成分を0にする
        for rr in range(dof_total):
            kc_mat[r, rr] = 0.0
        
        # 拘束条件の存在する列の成分を0にする
        for cc in range(dof_total):
            kc_mat[cc, r] = 0.0

        # 対角成分を1にする
        kc_mat[r, r] = 1
    
        # 変位拘束成分に対応する荷重ベクトルを修正
        f_vec[r] = u_vec[r]


In [ ]:
#連立方程式を解く(ガウス法)

u_vec_ans = np.linalg.solve(kc_mat, f_vec)

In [ ]:
# 変位解ベクトルの処理

u_x = None
u_y = None
u_ans = []

for i, a in enumerate(u_vec_ans):
    if i % 2 == 0:
        u_x = a
    else:
        u_y = a
        u_ans.append([u_x, u_y, 0])

df_u_ans = pd.DataFrame(u_ans, columns=["u_x", "u_y", "u_z"])

In [ ]:
# 節点反力を計算

f_ref_vec = np.zeros(dof_total)

for r in range(dof_total):
    for c in range(dof_total):
        f_ref_vec[r] += k_mat[c, r] * u_vec_ans[c]



# ひずみベクトルの計算
strain_vectors = []

for element_no in df_model["element_no"].tolist():
    node_1 = int(df_model[df_model["element_no"] == element_no]["tri_node_1"])
    node_2 = int(df_model[df_model["element_no"] == element_no]["tri_node_2"])
    node_3 = int(df_model[df_model["element_no"] == element_no]["tri_node_3"])

    strain_vector = b_mat[element_no] @ np.array([
        df_u_ans.iloc[node_1 - 1]["u_x"],
        df_u_ans.iloc[node_1 - 1]["u_y"],
        df_u_ans.iloc[node_2 - 1]["u_x"],
        df_u_ans.iloc[node_2 - 1]["u_y"],
        df_u_ans.iloc[node_3 - 1]["u_x"],
        df_u_ans.iloc[node_3 - 1]["u_y"],
    ])
    strain_vectors.append(strain_vector)
df_strain = pd.DataFrame(strain_vectors, columns=("strain_x", "strain_y", "strain_xy"))



# 応力ベクトルの計算
stress_vectors = []
for strain_vector in strain_vectors:
    stress_vector = d_mat @ strain_vector
    stress_vectors.append(stress_vector)
df_stress = pd.DataFrame(stress_vectors, columns=("stress_x", "stress_y", "stress_xy"))
    

In [ ]:
# vtkファイルで出力

# モデルをstructureとしてマウント
tri_polygons = (df_model[["tri_node_1", "tri_node_2", "tri_node_3"]].values - 1).tolist()
tri_nodes = df_nodes[["node_x", "node_y", "node_z"]].values
structure=pyvtk.PolyData(points=tri_nodes, polygons=tri_polygons)


# 変位をマウント
displacement_vector = pyvtk.Vectors(df_u_ans.values, name='DisplacementVectors')

# ひずみをマウント
celldata_x_strain = pyvtk.Scalars(df_strain["strain_x"].values, name='strain_x', lookup_table="default")
celldata_y_strain = pyvtk.Scalars(df_strain["strain_y"].values, name='strain_y', lookup_table="default")
celldata_xy_strain = pyvtk.Scalars(df_strain["strain_xy"].values, name='strain_xy', lookup_table="default")

# 応力をマウント
celldata_x_stress = pyvtk.Scalars(df_stress["stress_x"].values, name='stress_x', lookup_table="default")
celldata_y_stress = pyvtk.Scalars(df_stress["stress_y"].values, name='stress_y', lookup_table="default")
celldata_xy_stress = pyvtk.Scalars(df_stress["stress_xy"].values, name='stress_xy', lookup_table="default")

# 節点データをパッキング
point_data = pyvtk.PointData(
    displacement_vector
)

# 要素データをパッキング
cell_data = pyvtk.CellData(
    celldata_x_strain,
    celldata_y_strain,
    celldata_xy_strain,
    celldata_x_stress,
    celldata_y_stress,
    celldata_xy_stress
)

# vtk_dataとしてデータを作成
vtk_data = pyvtk.VtkData(structure, "# test", cell_data, point_data)

# 書き出し
vtk_data.tofile('tameshi')